Importation des packages 

In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.abspath("src"))
from agent import Actor, Critic, ReplayBuffer, DDPGAgent
from env import WirelessEnv
import torch
import rewards_computation
from rewards_computation import compute_rewards

Variables 

In [2]:
state_dim = 2
response_dim = 1
max_response = 10
Agent = DDPGAgent(state_dim, response_dim, max_response,buffer_size=1000)
env = WirelessEnv(num_antennas=1, num_users=1)

Vérification des objets

In [3]:
env_state = env._get_state()
print(env_state)
print(Agent)

{'users': [{'id': 0, 'pos': array([45.19224811, 85.2713231 ]), 'demand': 1}], 'antennas': [{'id': 0, 'pos': array([ 0, 50])}]}


In [4]:
demandes = env_state['users'][0]['demand']
print(demandes)

1


 Réponse de Agent à la demande de User. Par définition, la demande est une liste ou un ndarray de taille 2 : la puissance demandée par le user et la distance qui le sépare de l'antenne qui lui est assignée.

In [5]:
distances = env._user_antennas_distance()
print(distances)
# pour user_0
distance_utile = distances[0][0]
print(distance_utile)

{0: {0: 57.32717961162743}}
57.32717961162743


In [6]:
demandes = [demandes, distance_utile]
print(demandes)

[1, 57.32717961162743]


In [7]:
response = Agent.select_response(demandes) # attention la demande ici est une list ou un ndarray
print(response)

[10.]


Calcule de la recompense de l'antenne 0

In [8]:
response_dict = {0: response}
print(response_dict)

{0: array([10.], dtype=float32)}


In [9]:
reward = compute_rewards(response_dict, env)

In [10]:
print(reward)

{0: array([0.17144667], dtype=float32)}


In [11]:
print(demandes, response, reward)

[1, 57.32717961162743] [10.] {0: array([0.17144667], dtype=float32)}


In [12]:
batch_size = 60
for i in range(1000):
    response = Agent.select_response(demandes) 
    response_dict = {0: response}
    reward_dict = compute_rewards(response_dict,env)
    reward = reward_dict[0]
    if i%100 == 0:
        print([demandes, response, reward])
    Agent.buffer.add(demandes, response, reward, demandes)


[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]
[[1, 57.32717961162743], array([10.], dtype=float32), array([0.17144667], dtype=float32)]


In [13]:
Agent.buffer.size()

1000

Entrainement

In [14]:
Agent.train()

/home/ing/Logiciels/anaconda/envs/ift6758-conda-env/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([64, 64, 1])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [15]:
print(Agent)